# Projeto 1 - Ciência dos Dados

Nome: Diego Saragoza da Sila

Nome: Gabriel Ribeiro

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [9]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from cleantext import clean_words, clean

In [10]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\user\OneDrive - Insper - Institudo de Ensino e Pesquisa\S2\CDADOS\CDADOS-P1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [11]:
dados = pd.read_excel('Election.xlsx')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O nosso "produto" é as eleições de 2020 dos Estados Unidos da América. Durante nossas classificações, levamos em conta os nomes dos candidatos aos cargos e o contexto da frase do tweet em questão, caso houvesse alguma evidencia sobre votos ou eleições classificavamos este como relevante.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [54]:
#Pegando cada frase dos tweets, aplicando a função clean e transformando os dados em uma Series
tweets = []
for phrase in dados['Treinamento']:
    p = clean(phrase, punct = True, extra_spaces = True)
    tweets.append(p)
            
tweets = pd.Series(tweets)
print(tweets)

0      biden aims for big michigan win while sanders ...
1      wowits almost like berniesanders has won the e...
2      rt pupiczech2 im trying to gather stuff to sta...
3      annwilburn7 dnc sorry some of us think for our...
4      rt googleexpertuk handyoscar really do think i...
                             ...                        
495    rt justplainanon pattyangel64 koigi3 disagree ...
496    rt reedgalen corygardner is unavailable for co...
497    leftistbitch666 houndsofearth thomasisonline g...
498    rt reptdemocracy in slovakia’s recent election...
499    realdonaldtrump well if we learned anything fr...
Length: 500, dtype: object


In [104]:
#P(A|B) = Probabilidade de relevancia da frase dado suas palavras
#P(B|A) = Probabilidade das palavras da frase acontecerem dado suas relevancias
#P(A)   = Probabilidade de palavras relevantes
#P(B)   = Probabilidade da palavra acontecer

#Número de frases relevantes
r = dados['Relevancia'] == 1
relevantes = dados.loc[r, 'Relevancia']

#Tabela relativa de relevantes
tabela_relevantes = dados.loc[r, 'Treinamento']
rel = ''
for phrase in tabela_relevantes:
    for word in phrase.split():
        rel += ' ' + word
        
rel = clean_words(rel, punct = True, extra_spaces = True)
tabela_relativa_relevantes = pd.Series(rel)
tabela_relativa_relevantes = tabela_relativa_relevantes.value_counts(True)

#Probabilidade de frases relevantes
probR = len(relevantes) / len(tweets)

#Número de frases não relevantes
r = dados['Relevancia'] == 0
naorelevantes = dados.loc[r, 'Relevancia']

#Tabela relativa de não relevantes
tabela_nrelevantes = dados.loc[r, 'Treinamento']
nrel = ''
for phrase in tabela_nrelevantes:
    for word in phrase.split():
        nrel += ' ' + word
        
nrel = clean_words(nrel, punct = True, extra_spaces = True)
tabela_relativa_nrelevantes = pd.Series(nrel)
tabela_relativa_nrelevantes = tabela_relativa_nrelevantes.value_counts(True)

#Probabilidade de frases relevantes
probNR = len(naorelevantes) / len(tweets)

In [110]:
#Calculando a probabilidade das frases serem relevantes ou não usando o teorema de Bayes
probFraseDadoR  = 1
probFraseDadoNR = 1
tabela_prob_relevante = []
tabela_prob_nrelevante = []

#Calculando P(B|A) para relevantes
for phrase in tweets:
    for word in phrase.split():
        try:
            probFraseDadoR *= tabela_relativa_relevantes[word]
        except:
            pass
        
    tabela_prob_relevante.append(probFraseDadoR)
    probFraseDadoR = 1

#Calculando P(B|A) para não relevantes
for phrase in tweets:
    for word in phrase.split():
        try:
            probFraseDadoR *= tabela_relativa_nrelevantes[word]
        except:
            pass
        
    tabela_prob_nrelevante.append(probFraseDadoR)
    probFraseDadoR = 1

#Calculando P(A|B)
probRelevanteDadoF = []
for value in tabela_prob_relevante:
    probRelevanteDadoF.append(value * probR)

probNRelevanteDadoF = []
for value in tabela_prob_nrelevante:
    probNRelevanteDadoF.append(value * probNR)

Teste = []
relevantes = 0
naorelevantes = 0
acertos = 0
for i in range(len(probRelevanteDadoF)):
    if probRelevanteDadoF[i] > probNRelevanteDadoF[i]:
        Teste.append(1)
        relevantes += 1
    else:
        Teste.append(0)
        naorelevantes += 1
    
    #print(Teste[len(Teste) - 1], dados.at[i, 'Relevancia'])
    if Teste[len(Teste) - 1] == dados.at[i, 'Relevancia']:
        acertos += 1
        
print(acertos)

1.5322515290444705e-49 1.8471136977810412e-34
1.2938861164958172e-25 2.0631690095278577e-16
6.886982777455295e-42 5.131422308023097e-74
6.184480326251329e-69 6.249891568268519e-56
1.3353193631059444e-54 4.716567339143681e-77
2.661094154763539e-37 8.774984286277743e-77
1.8275796347573603e-31 6.577702599537651e-13
1.612839568940555e-62 3.9332489799454766e-40
1.9258009907517357e-46 6.136120758861219e-65
3.4260052615200167e-70 7.597346767637968e-99
6.581550935995912e-51 7.687790851653939e-63
1.6723938903966784e-43 3.371327220275045e-36
2.8517014912310155e-54 7.610155120229286e-80
4.946225356215863e-90 2.85491130873638e-66
5.4521505672518554e-30 3.555524067111778e-64
3.3320375805198164e-33 1.4650762722579813e-62
1.5995137931781556e-52 2.0433081008392878e-70
2.897093106529627e-102 3.9636000858034704e-135
1.1804028929443754e-42 1.1564531969492364e-72
5.912402564145933e-32 2.36266302633081e-66
3.9812250430117387e-69 3.537284482405495e-44
6.54754265535256e-47 3.331331435686488e-32
1.37747187136

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**